In [1]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
from anndata import AnnData, read_csv
import seaborn as sns
from matplotlib import pyplot as plt
from glob import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
loc = './data/'

# Create Zebrafish AnnData

### Assign annotations from SAMap

Download the raw zebrafish h5ad File and map annotations from [elife paper](https://elifesciences.org/articles/66747).

In [16]:
!wget -O ./data/WagnerScience2018.h5ad https://kleintools.hms.harvard.edu/paper_websites/wagner_zebrafish_timecourse2018/WagnerScience2018.h5ad 

--2022-11-07 14:34:27--  https://kleintools.hms.harvard.edu/paper_websites/wagner_zebrafish_timecourse2018/WagnerScience2018.h5ad
Resolving kleintools.hms.harvard.edu (kleintools.hms.harvard.edu)... 134.174.159.103
Connecting to kleintools.hms.harvard.edu (kleintools.hms.harvard.edu)|134.174.159.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 258580699 (247M)
Saving to: ‘./data/WagnerScience2018.h5ad’

./data/WagnerScienc 100%[===================>] 246.60M  5.53MB/s    in 28s     

2022-11-07 14:34:56 (8.81 MB/s) - ‘./data/WagnerScience2018.h5ad’ saved [258580699/258580699]



In [17]:
zebrafish = sc.read(os.path.join(loc,'WagnerScience2018.h5ad'))
zebrafish.obs['cluster'] = pd.Categorical([z[6:] if '-' in z else z for z in zebrafish.obs['ClusterName']])

In [19]:
with open(os.path.join(loc,'zebrafish_cell_types_mapping')) as f:
    cell_types_mapping = f.readlines()
ct_map = {}
for line in cell_types_mapping[1:]:
    el = line.split("\t")
    ct_map[el[0].strip()] = el[1].strip()
ct_map['periderm'] = 'Periderm'
ct_map['pluripotent'] = 'Pluripotent'
ct_map['neural - floorplate posterior'] = 'Notoplate'
ct_map['neural crest - mcamb'] = 'Neural crest'
ct_map['neural crest - melanoblast'] = 'Neural crest'
ct_map['neural crest - iridoblast'] = 'Neural crest'
ct_map['neural crest - xanthophore'] = 'Neural crest'
ct_map['neural crest - crestin'] = 'Neural crest'

In [21]:
samap_clusters = []
not_found = []
for f in zebrafish.obs['cluster']:
    if f.strip() not in ct_map:
        samap_clusters.append('NaN')
        not_found.append(f)
    else:
        samap_clusters.append(ct_map[f.strip()])
print(set(not_found))
zebrafish.obs['cell_type'] = pd.Categorical(samap_clusters)
zebrafish = zebrafish[zebrafish.obs['cell_type']!='NaN']
zebrafish.write(os.path.join(loc,'zebrafish_annot.h5ad'))

{'EVL', 'NaN'}


### Load zebrafish data

In [22]:
zebrafish = sc.read(os.path.join(loc,'zebrafish_annot.h5ad'))

In [23]:
zebrafish.obs

,n_counts,unique_cell_id,cell_names,library_id,batch,ClusterID,ClusterName,TissueID,TissueName,TimeID,cluster,cell_type
index,,,,,,,,,,,,
0-0-0,15111.0,DEW050_AGTCAATAC-TTGGATCG,bcGPGV,DEW050,0,1,04hpf-pluripotent,9,Pluripotent,4hpf,pluripotent,Pluripotent
1-0-0,2337.0,DEW050_AAGAACGGG-GCGTTGCT,bcDSDI,DEW050,0,1,04hpf-pluripotent,9,Pluripotent,4hpf,pluripotent,Pluripotent
2-0-0,2078.0,DEW050_GACCTACTAG-TTAGTCCG,bcENHV,DEW050,0,1,04hpf-pluripotent,9,Pluripotent,4hpf,pluripotent,Pluripotent
3-0-0,1648.0,DEW050_GTTTGTTT-GGTCCCTT,bcAABE,DEW050,0,1,04hpf-pluripotent,9,Pluripotent,4hpf,pluripotent,Pluripotent
4-0-0,1153.0,DEW050_TGATTGCACGC-TAACCATC,bcFTTU,DEW050,0,1,04hpf-pluripotent,9,Pluripotent,4hpf,pluripotent,Pluripotent
...,...,...,...,...,...,...,...,...,...,...,...,...
1993-28-6,2525.0,DEW169_ATTTCCAT-CAGTCCCT,bcDANY,DEW169,6,135,24hpf-muscle - myl1,8,Mesoderm,24hpf,muscle - myl1,Skeletal muscle
1994-28-6,1548.0,DEW169_CTACGGGA-ATACCCAG,bcELQP,DEW169,6,157,24hpf-differentiating neurons - eomesa,1,Forebrain / Optic,24hpf,differentiating neurons - eomesa,Neuron
2000-28-6,5054.0,DEW169_TGGAAAGC-CCGCAACT,bcIGIF,DEW169,6,138,24hpf-neural - diencephalon,1,Forebrain / Optic,24hpf,neural - diencephalon,Forebrain/midbrain


In [24]:
zebrafish.obs_names = [x for x in zebrafish.obs_names ]
zebrafish.var_names = [x for x in zebrafish.var_names ]
zebrafish.obs.cell_type = [x for x in zebrafish.obs.cell_type ]

In [25]:
sc.pp.filter_cells(zebrafish, min_genes=500)
sc.pp.filter_genes(zebrafish, min_cells=10)

In [26]:
zebrafish.X.toarray().max()

3678.0

In [27]:
zebrafish.write(os.path.join(loc, "zebrafish.h5ad"))

In [30]:
zebrafish

AnnData object with n_obs × n_vars = 63371 × 30032
    obs: 'n_counts', 'unique_cell_id', 'cell_names', 'library_id', 'batch', 'ClusterID', 'ClusterName', 'TissueID', 'TissueName', 'TimeID', 'cluster', 'cell_type', 'n_genes'
    var: 'n_cells'

# Create Frog AnnData
Download the raw frog h5ad File and map annotations from [elife paper](https://elifesciences.org/articles/66747).

Download the Data from GSE

In [29]:
!wget -O ./data/GSE113074_Raw_combined.annotated_counts.tsv.gz "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE113nnn/GSE113074/suppl/GSE113074_Raw_combined.annotated_counts.tsv.gz"

--2022-11-07 14:41:36--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE113nnn/GSE113074/suppl/GSE113074_Raw_combined.annotated_counts.tsv.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 165.112.9.230, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177842456 (170M) [application/x-gzip]
Saving to: ‘./data/GSE113074_Raw_combined.annotated_counts.tsv’

./data/GSE113074_Ra 100%[===================>] 169.60M  9.61MB/s    in 20s     

2022-11-07 14:41:57 (8.52 MB/s) - ‘./data/GSE113074_Raw_combined.annotated_counts.tsv’ saved [177842456/177842456]



In [37]:
!gunzip ./data/GSE113074_Raw_combined.annotated_counts.tsv.gz

In [19]:
def generate_frog_h5ad():
    filepath_frog = os.path.join(loc,'GSE113074_Raw_combined.annotated_counts.tsv')
    with open(filepath_frog) as f:
        frog_data = f.readlines()
    barcodes = [f for f in frog_data[5].split("\t")]
    libraries = [f.strip() for f in frog_data[3].split("\t")]
    parent_clusters = [f[4:].strip() if f.startswith('S') else f for f in frog_data[8].split("\t")]
    clusters = [f[4:].strip() if f.startswith('S') else f for f in frog_data[7].split("\t")]
    dev_stage = [f.strip() for f in frog_data[6].split("\t")]
    genes = []
    data = np.zeros((len(frog_data[9:]),len(barcodes)-1))
    for i, f in enumerate(frog_data[9:]):
        line = f.split("\t")
        genes.append(line[0])
        data[i, :] = [np.float(l) for j,l in enumerate(line[1:])]
    
    # create anndata
    adata = AnnData(data.transpose())
    adata.var_names = genes
    adata.obs_names = barcodes[1:]
    adata.obs["library"] = libraries[1:]
    adata.obs['clusters'] = clusters[1:]
    adata.obs['dev_stage'] = dev_stage[1:]
    adata.obs['parent_clusters'] = parent_clusters[1:]
    # add samap cell type annotations
    with open(os.path.join(loc, 'frog_cell_types_mapping')) as f:
        cell_types_mapping = f.readlines()
    ct_map = {}
    for line in cell_types_mapping[1:]:
        el = line.split("\t")
        ct_map[el[0].strip()] = el[1].strip()
    ct_map['Outlier'] = 'Outlier'
    samap_clusters = []
    for f in adata.obs['clusters']:
        if f.strip() not in ct_map:
            print(f)
        samap_clusters.append(ct_map[f.strip()])
    adata.obs['cell_type'] = samap_clusters
    
    adata.write(os.path.join(loc,'GSE113074_Corrected_combined.annotated_counts.h5ad'))
    return adata

In [20]:
frog = generate_frog_h5ad()

In [21]:
frog_annot = sc.read_h5ad(os.path.join(loc,'GSE113074_Corrected_combined.annotated_counts.h5ad'))

In [22]:
frog_counts = frog

In [23]:
frog_annot

AnnData object with n_obs × n_vars = 136966 × 26550
    obs: 'library', 'clusters', 'dev_stage', 'parent_clusters', 'cell_type'

In [25]:
frog = frog[frog.obs['cell_type']!='Outlier']

In [26]:
frog.obs_names = [x for x in frog.obs_names ]
frog.var_names = [x for x in frog.var_names ]
frog.obs.cell_type = [x for x in frog.obs.cell_type ]

In [27]:
sc.pp.filter_cells(frog, min_genes=500)
sc.pp.filter_genes(frog, min_cells=10)

In [28]:
frog.X

array([[ 4.,  1.,  0., ...,  2.,  0., 10.],
       [ 1.,  0.,  0., ...,  0.,  0.,  5.],
       [ 1.,  7.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [29]:
frog.write(os.path.join(loc, "frog.h5ad"))